IBM Data Science Capstone course.
This notebook is for part 1 of week 3 assignment.
The Wikipedia page with Toronto's list of postal codes, boroughs and neighborhoods is scraped and a dataframe is created.

In [3]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

I used BeautifulSoup to scrape the Wikipedia page. I inspected the html code of the page to find the table and its class.

In [4]:
# get wikipedia page html
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')

#print(soup)
#after inspecting html of page, the class of the table is found
table = soup.find('table', {"class": "wikitable sortable"})

I isolated the rows of the table using their tags (tr) and then found the relevant data in the columns (td). I created a dataframe and then cleaned it up by removing the rows that don't have a borough, assigning the borough's name to neighborhoods that don't have a name and joining the rows that have the same borough's name.

In [5]:
rows=table.find_all('tr') #all table's rows

# find name of columns from first row of table
col_hs=rows[0].find_all('th') #columns name have tag <th
col_names=[colh.text.rstrip('\n') for colh in col_hs]
table_data=[]
for row in rows[1:]:
    col_data=row.find_all('td') #table elements have tag <td
    col_data=[col.text.rstrip('\n') for col in col_data]
    table_data.append([cell_data for cell_data in col_data])
table_df = pd.DataFrame(table_data, columns=col_names) #create dataframe

# Remove rows that don't have an assigned borough
table_df1=table_df[table_df['Borough']!='Not assigned'].reset_index().drop(['index'],axis=1)

# Assign borough's name to neighborhoods that don't have a name
table_df1['Neighbourhood'][table_df1['Neighbourhood']=='Not assigned']=table_df1['Borough']

# list of unique post codes and corresponding boroughs
df2=table_df1.drop_duplicates(subset=['Postcode','Borough']).sort_values(by='Postcode',axis=0,ascending=True)
postc_list=df2['Postcode']
boro_list=df2['Borough']

neigh_list=[]
s=', ' #delimiter
for pc in postc_list:
    neigh_list.append(s.join(table_df1[table_df1['Postcode']==pc]['Neighbourhood']))

newdf=pd.DataFrame({'PostalCode':postc_list, 'Borough': boro_list,'Neighborhood': neigh_list}).reset_index(drop=True)
newdf.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
print('The new dataframe has {} rows'.format(newdf.shape[0]))

The new dataframe has 103 rows
